# Exercise - Pricing Swaptions


#### Notation Commands

$$\newcommand{\Black}{\mathcal{B}}
\newcommand{\Blackcall}{\Black_{\mathrm{call}}}
\newcommand{\Blackput}{\Black_{\mathrm{put}}}
\newcommand{\EcondS}{\hat{S}_{\mathrm{conditional}}}
\newcommand{\Efwd}{\mathbb{E}^{T}}
\newcommand{\Ern}{\mathbb{E}^{\mathbb{Q}}}
\newcommand{\Tfwd}{T_{\mathrm{fwd}}}
\newcommand{\Tunder}{T_{\mathrm{bond}}}
\newcommand{\accint}{A}
\newcommand{\carry}{\widetilde{\cpn}}
\newcommand{\cashflow}{C}
\newcommand{\convert}{\phi}
\newcommand{\cpn}{c}
\newcommand{\ctd}{\mathrm{CTD}}
\newcommand{\disc}{Z}
\newcommand{\done}{d_{1}}
\newcommand{\dt}{\Delta t}
\newcommand{\dtwo}{d_{2}}
\newcommand{\flatvol}{\sigma_{\mathrm{flat}}}
\newcommand{\flatvolT}{\sigma_{\mathrm{flat},T}}
\newcommand{\float}{\mathrm{flt}}
\newcommand{\freq}{m}
\newcommand{\futprice}{\mathcal{F}(t,T)}
\newcommand{\futpriceDT}{\mathcal{F}(t+h,T)}
\newcommand{\futpriceT}{\mathcal{F}(T,T)}
\newcommand{\futrate}{\mathscr{f}}
\newcommand{\fwdprice}{F(t,T)}
\newcommand{\fwdpriceDT}{F(t+h,T)}
\newcommand{\fwdpriceT}{F(T,T)}
\newcommand{\fwdrate}{f}
\newcommand{\fwdvol}{\sigma_{\mathrm{fwd}}}
\newcommand{\fwdvolTi}{\sigma_{\mathrm{fwd},T_i}}
\newcommand{\grossbasis}{B}
\newcommand{\hedge}{\Delta}
\newcommand{\ivol}{\sigma_{\mathrm{imp}}}
\newcommand{\logprice}{p}
\newcommand{\logyield}{y}
\newcommand{\mat}{(n)}
\newcommand{\nargcond}{d_{1}}
\newcommand{\nargexer}{d_{2}}
\newcommand{\netbasis}{\tilde{\grossbasis}}
\newcommand{\normcdf}{\mathcal{N}}
\newcommand{\notional}{K}
\newcommand{\pfwd}{P_{\mathrm{fwd}}}
\newcommand{\pnl}{\Pi}
\newcommand{\price}{P}
\newcommand{\probexer}{\hat{\mathcal{P}}_{\mathrm{exercise}}}
\newcommand{\pvstrike}{K^*}
\newcommand{\refrate}{r^{\mathrm{ref}}}
\newcommand{\rrepo}{r^{\mathrm{repo}}}
\newcommand{\spotrate}{r}
\newcommand{\spread}{s}
\newcommand{\strike}{K}
\newcommand{\swap}{\mathrm{sw}}
\newcommand{\swaprate}{\cpn_{\swap}}
\newcommand{\tbond}{\mathrm{fix}}
\newcommand{\ttm}{\tau}
\newcommand{\value}{V}
\newcommand{\vega}{\nu}
\newcommand{\years}{\tau}
\newcommand{\yearsACT}{\tau_{\mathrm{act/360}}}
\newcommand{\yield}{Y}$$


# 1. Pricing the Swaption


## Swaption Vol Data

The file `data/swaption_vol_data_2025-06-30.xlsx` has market data on the implied volatility skews for swaptions. Note that it has several columns:
* `expry`: expiration of the swaption
* `tenor`: tenor of the underlying swap
* `model`: the model by which the volatility is quoted. (All are Black.)
* `-200`, `-100`, etc.: The strike listed as difference from ATM strike (bps). Note that ATM is considered to be the **forward swapa rate** which you can calculate.


Your data: you will use a single row of this data for the `1x4` swaption.
* date: `2025-06-30`
* expiration: 1yr
* tenor: 4yrs


## Rate Data

The file `data/cap_curves_2025-06-30.xlsx` gives 
* SOFR swap rates, 
* their associated discount factors
* their associated forward interest rates.

You will not need the cap data (flat or forward vols) for this problem.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import log, sqrt, erf

VOL_FILE = 'swaption_vol_data_2025-06-30.xlsx'
RATE_FILE = 'cap_curves_2025-06-30.xlsx'
vol_data = pd.read_excel(VOL_FILE)
rate_data = pd.read_excel(RATE_FILE)
display(vol_data.head(5))
display(rate_data.head(5))

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
0,SOFR,swaption,black,2025-06-30,1,1,72.250,46.870,39.100,36.000,33.39,31.300,29.760,28.17,28.660
1,SOFR,swaption,black,2025-06-30,1,2,65.780,44.400,37.970,35.460,33.39,31.750,30.530,29.19,29.300
2,SOFR,swaption,black,2025-06-30,1,3,57.870,40.610,35.560,33.650,32.11,30.920,30.060,29.14,29.290
3,SOFR,swaption,black,2025-06-30,1,4,54.405,38.565,33.925,32.195,30.83,29.805,29.095,28.43,28.885
4,SOFR,swaption,black,2025-06-30,1,5,50.940,36.520,32.290,30.740,29.55,28.690,28.130,27.72,28.480


,tenor,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
0,0.25,0.042353,0.042353,0.989523,NaN,NaN,NaN
1,0.50,0.040859,0.040852,0.979883,0.039351,0.156842,0.156842
2,0.75,0.039391,0.039372,0.971043,0.036414,0.180709,0.201708
3,1.00,0.038115,0.038083,0.962807,0.034217,0.204576,0.240464
4,1.25,0.036704,0.036653,0.955417,0.030938,0.242127,0.328341


## The Swaption

Consider the following swaption with the following features:
* underlying is a fixed-for-floating (SOFR) swap
* the underlying swap has **quarterly** payment frequency
* this is a **payer** swaption, which gives the holder the option to **pay** the fixed swap rate and receive SOFR.


### 1.1
Calculate the (relevant) forward swap rate. That is, the one-year forward 4-year swap rate.

### 1.2
Price the swaptions at the quoted implied volatilites and corresponding strikes, all using the just-calculated forward swap rate as the underlying.

### 1.3
To consider how the expiration and tenor matter, calculate the prices of a few other swaptions for comparison. 
* No need to get other implied vol quotes--just use the ATM implied vol you have for the swaption above. (Here we are just interested in how Black's formula changes with changes in tenor and expiration.)
* No need to calculate for all the strikes--just do the ATM strike.

Alternate swaptions
* The 3mo x 4yr swaption
* The 2yr x 4yr swaption
* the 1yr x 2yr swaption

Report these values and compare them to the price of the `1y x 4y` swaption.


## 1.1

In [9]:
def interp_df(rate_data, t):
    "Linear interpolation for discount factors"
    x = rate_data['tenor'].to_numpy(dtype=float)
    y = rate_data['discounts'].to_numpy(dtype=float)
    return np.interp(t, x, y)

def fwd_swap_rate_and_annuity(rate_data, start=1.0, tenor=4.0, freq=4):
    "Calculate forward swap rate for given start, tenor, and frequency"
    dt = 1/freq
    end = start + tenor
    #fixed payment dates of underlying fwd starting swap
    pay_times = np.arange(start+dt, end+dt, dt)
    P_start = interp_df(rate_data, start)
    P_end = interp_df(rate_data, end)

    annuity = np.sum(dt * interp_df(rate_data, pay_times))
    fwd_rate = (P_start - P_end) / annuity

    return fwd_rate, annuity, P_start, P_end

fwd_1x4_res = fwd_swap_rate_and_annuity(rate_data, start=1.0, tenor=4.0, freq=4)
fwd_1x4, annuity_1x4, P_start_1x4, P_end_1x4 = fwd_1x4_res
print(f"Forward swap rate for 1x4 swap: {fwd_1x4:.4%}")

Forward swap rate for 1x4 swap: 3.2698%


## 1.2

In [ ]:
def norm_cdf(x): return 0.5 * (1.0 + erf(x / sqrt(2.0)))

def black76_call(F, K, sigma, T):
    "Black76 call option value on a forward (undiscounted)"
    if sigma<=0 or T<=0:
        return max(F-K, 0.0)
    vol_sqrt = sigma * sqrt(T)
    d1 = (log(F/K) + 0.5*vol_sqrt**2) / vol_sqrt
    d2 = d1 - vol_sqrt
    call_price = F * norm_cdf(d1) - K * norm_cdf(d2)
    return call_price

row_1x4 = vol_data.query('expiration == 1 and tenor ==4').iloc[0]
strike_cols = [c for c in vol_data.columns if str(c).lstrip("-").isdigit()]
strike_cols = sorted(strike_cols, key = lambda x: int(x))

F = fwd_1x4 
A = annuity_1x4 #annuity = PV of $1 per year paid on the fixed leg of the swap
Texp = 1.0

rows = []
for c in strike_cols:
    dk_bps = int(c)
    K = F + dk_bps/10000
    iv = float(row_1x4[c]) / 100.0
    black = black76_call(F, K, iv, Texp)
    price = black * A
    rows.append({
        'dk (bps)': dk_bps,
        'Strike': K,
        'Implied Vol (%)': iv * 100,
        'Black (F,K)': black,
        'Swaption PV (per $1 notional)': price
    })
swaption_prices = pd.DataFrame(rows)
display(swaption_prices.round(4))


,dk (bps),Strike,Implied Vol (%),"Black (F,K)",Swaption PV (per $1 notional)
0,-200,0.0127,54.405,0.0202,0.0727
1,-100,0.0227,38.565,0.0110,0.0395
2,-50,0.0277,33.925,0.0070,0.0254
3,-25,0.0302,32.195,0.0054,0.0194
4,0,0.0327,30.830,0.0040,0.0144
5,25,0.0352,29.805,0.0029,0.0104
6,50,0.0377,29.095,0.0020,0.0074
7,100,0.0427,28.430,0.0010,0.0036
8,200,0.0527,28.885,0.0002,0.0009


## 1.3

In [16]:
targets = [
    (0.25, 4.0),
    (2.00, 4.0),
    (1.00, 2.0),
    (1.00, 4.0) # baseline for comparison
]

#ATM volatility for 1x4 swaption (from vol data)
sigma_atm = float(row_1x4[0]) / 100.0

def price_atm_payer_swaption(rate_data, expiry, tenor, sigma=sigma_atm, freq=4, notional=100.0):
    F, A, P_start, P_end = fwd_swap_rate_and_annuity(rate_data, start=expiry, 
                                                     tenor=tenor, freq=freq)
    K = F
    pv_per_1 = A * black76_call(F, K, sigma, expiry)
    pv = pv_per_1 * notional
    out = {
        'expiry (y)': expiry,
        'tenor (y)': tenor,
        'forward swap rate': F,
        'ATM vol (fixed)': sigma,
        'annuity': A,
        'swaption PV (per $100 notional)': pv
    }
    return out

output_df = pd.DataFrame([price_atm_payer_swaption(rate_data, e, t) for e, t in targets])
base = output_df.loc[(output_df['expiry (y)'] == 1.0) & (output_df['tenor (y)'] == 4.0)].iloc[0]
output_df["PV / PV(1x4)"] = output_df['swaption PV (per $100 notional)'] / base['swaption PV (per $100 notional)']

display(output_df.round(4))

,expiry (y),tenor (y),forward swap rate,ATM vol (fixed),annuity,swaption PV (per $100 notional),PV / PV(1x4)
0,0.25,4.0,0.0330,0.3083,3.6922,0.7485,0.5186
1,2.00,4.0,0.0343,0.3083,3.4845,2.0599,1.4272
2,1.00,2.0,0.0312,0.3083,1.8605,0.7108,0.4924
3,1.00,4.0,0.0327,0.3083,3.6032,1.4434,1.0000
